In [1]:
import conllutils as cu

# Loading the different tree banks

In [3]:
talbanken_path = '/home/norrman/GitHub/UD_Swedish-Talbanken/not-to-release/output/temp/sv6.conllu'
pud_path = '/home/norrman/GitHub/UD_Swedish-PUD/sv_pud-ud-test-updated.conllu'
lines_path_train = '/home/norrman/GitHub/UD_Swedish-LinES/sv_lines-ud-train-updated.conllu'
lines_path_dev = '/home/norrman/GitHub/UD_Swedish-LinES/sv_lines-ud-dev-updated.conllu'
lines_path_test = '/home/norrman/GitHub/UD_Swedish-LinES/sv_lines-ud-test-updated.conllu'

talbanken_data = cu.ConlluTreebank.load_conllu_from_file(talbanken_path)
pud_data = cu.ConlluTreebank.load_conllu_from_file(pud_path)
lines_data_train = cu.ConlluTreebank.load_conllu_from_file(lines_path_train)
lines_data_dev = cu.ConlluTreebank.load_conllu_from_file(lines_path_dev)
lines_data_test = cu.ConlluTreebank.load_conllu_from_file(lines_path_test)

treebanks = {'talbanken': talbanken_data, 'pud': pud_data, 'lines_dev': lines_data_dev, 'lines_test': lines_data_test, 'lines_test': lines_data_train}

In [ ]:
# pud_data.write_to_file('/home/norrman/GitHub/UD_Swedish-PUD/regen_check.conllu')
# lines_data_train.write_to_file('/home/norrman/GitHub/UD_Swedish-LinES/regen_check_train.conllu')
# lines_data_dev.write_to_file('/home/norrman/GitHub/UD_Swedish-LinES/regen_check_dev.conllu')
# lines_data_test.write_to_file('/home/norrman/GitHub/UD_Swedish-LinES/regen_check_test.conllu')

# Missed ADJ

The following two functions aims to extract any adj that is not caught by our previous fixes and write them to separate documents for manual inspection.

In [13]:
def criteria(token):
    if (token.lemma[-1] in ['f', 'g', 'l', 'm', 'n', 'p', 'r', 's', 'v']) and (token.form == token.lemma+'t'):
        return False
        
    elif token.form[-2:] == 'ad' and token.lemma.endswith('a'):
        return False
        
    elif (token.form[-3:] == 'ade') and ((token.lemma.endswith('a')) or (token.lemma.endswith('d'))):
        return False
    
    elif token.form[-2:] == 'at' and ((token.lemma.endswith('a')) or (token.lemma.endswith('d'))):
        return False

    elif token.form[-3:] == 'sta' and token.feats.get('Degree', None) == 'Sup':
        return False
    
    elif token.form[-2:] == 'st' and token.feats.get('Degree', None) == 'Sup':
        return False
        
    elif token.form[-2:] == 're' and token.feats.get('Degree', None) == 'Cmp':
        return False

    elif token.form[-3:] != 'sta' and token.form == token.lemma + 'a' and token.feats.get('Definite', None) == 'Def':
        return False
    
    elif token.form[-3:] != 'sta' and token.form == token.lemma + 'a' and token.feats.get('Number', None) == 'Plur':
        return False

    elif token.xpos is not None and ('RO' in token.xpos or token.xpos == 'ORD'):
        return False

    elif token.form[-3:] == 'nde':
        return False

    else:
        return True
    
def missed_adj(treebanks, outdir=''):

    if isinstance(treebanks, list):
        found_tokens = []
        for tb in treebanks:
            found_tokens.extend([(tok, None) for tok in tb.get_token_matches(upos='ADJ', func=criteria)])
    elif isinstance(treebanks, dict):
        found_tokens = []
        for key, tb in treebanks.items():
            found_tokens.extend([(tok, key) for tok in tb.get_token_matches(upos='ADJ', func=criteria)])
    else:
        found_tokens = [(tok, None) for tok in treebanks.get_token_matches(upos='ADJ', func=criteria)]

    with open(outdir+'adj-types-missed.tsv', 'w') as f:
        relevant_toks = sorted(list(set([(tok.form, tok.lemma, tok.reconstruct_feats()) for tok, key in found_tokens if tok.form != tok.lemma])), key=lambda a: a[0][::-1])
        
        for form, lemma, feats in relevant_toks:
            text = ''
            in_treebanks = set()

            for tok, key in found_tokens:
                if tok.form == form and tok.lemma == lemma and tok.reconstruct_feats() == feats:
                    text = tok.sent.text if not text else text
                    in_treebanks.add(key)
                    
            f.write(f"{form=}\t{lemma=}\t{feats}\t{in_treebanks=}\t{text}\n")

    with open(outdir+'ident-adj-types-missed.tsv', 'w') as f:
        relevant_toks = sorted(list(set([(tok.form, tok.lemma, tok.reconstruct_feats()) for tok, key in found_tokens if tok.form == tok.lemma])), key=lambda a: a[0][::-1])
        
        for form, lemma, feats in relevant_toks:
            text = ''
            in_treebanks = set()

            for tok, key in found_tokens:
                if tok.form == form and tok.lemma == lemma and tok.reconstruct_feats() == feats:
                    text = tok.sent.text if not text else text
                    in_treebanks.add(key)
                    
            f.write(f"{form=}\t{lemma=}\t{feats}\t{in_treebanks=}\t{text}\n")

# Checking for missed ADJ

In [9]:
talbanken_outdir = '/home/norrman/GitHub/UD_Swedish-Talbanken/not-to-release/sv_talbanken-'
pud_outdir = '/home/norrman/GitHub/UD_Swedish-PUD/sv_pud-'
lines_outdir = '/home/norrman/GitHub/UD_Swedish-LinES/sv_lines-'
combined_outdir = '/home/norrman/GitHub/UD_Swedish-Talbanken/not-to-release/combined-talbanken-pud-lines-'



missed_adj(talbanken_data, talbanken_outdir)
missed_adj(pud_data, pud_outdir)
missed_adj([lines_data_train, lines_data_dev, lines_data_test], lines_outdir)
missed_adj(treebanks, combined_outdir)





NameError: name 'missed_adj' is not defined

# Headless Tokens

In [4]:
headless_tokens = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(func=lambda a: a.head is None)]
for tok in headless_tokens:
    print(tok.sent_id, tok.to_conllu())

sv-ud-train-166 15.1	betalar	betala	VERB	VB|PRS|AKT	Mood=Ind|Tense=Pres|VerbForm=Fin|Voice=Act	_	_	4:ccomp|7:conj	_
sv-ud-train-166 23.1	betalar	betala	VERB	VB|PRS|AKT	Mood=Ind|Tense=Pres|VerbForm=Fin|Voice=Act	_	_	4:ccomp|7:conj	_
sv-ud-train-253 15.1	vistats	vistas	VERB	VB|SUP|SFO	VerbForm=Sup	_	_	9:conj:och	_
sv-ud-test-177 13.1	beräknas	beräkna	VERB	VB|PRS|SFO	Mood=Ind|Tense=Pres|VerbForm=Fin|Voice=Pass	_	_	2:conj	_
sv-ud-test-177 13.2	sjuka	sjuk	ADJ	JJ|POS|UTR/NEU|PLU|IND/DEF|NOM	Case=Nom|Definite=Ind|Degree=Pos|Number=Plur	_	_	13.1:xcomp	_
sv-ud-train-470 11.1	förekommer	förekomma	VERB	VB|PRS|AKT	Mood=Ind|Tense=Pres|VerbForm=Fin|Voice=Act	_	_	2:conj	_
sv-ud-train-518 13.1	få	få	VERB	VB|INF|AKT	VerbForm=Inf|Voice=Act	_	_	4:conj:men	_
sv-ud-train-547 6.1	Se	se	VERB	VB|IMP|AKT	Mood=Imp|VerbForm=Fin|Voice=Act	_	_	1:conj:och	_
sv-ud-train-667 6.1	Se	se	VERB	VB|IMP|AKT	Mood=Imp|VerbForm=Fin|Voice=Act	_	_	1:conj:och	_
sv-ud-test-207 5.1	Se	se	VERB	VB|IMP|AKT	Mood=Imp|VerbForm=Fin|Voice=

# Superlative ending with 'ste'

In [5]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(Degree='Sup',
                                                                                  func1=lambda tok: tok.form.endswith('ste'))]

feats = set([tok.reconstruct_feats() for tok in matches])
feats

{'Case=Nom|Definite=Def|Degree=Sup', 'Case=Nom|Definite=Ind|Degree=Sup'}

In [6]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(Degree='Sup',
                                                                                  Definite='Ind',
                                                                                  func1=lambda tok: tok.form.endswith('ste'))]
print(*list(set([tok.form for tok in matches])), sep='\n')
print(*list(set([tok.sent_id for tok in matches])), sep='\n')
print(*list(set([tok.sent.text for tok in matches])), sep='\n')



närmaste
vanligaste
Käraste
endaste
sv_lines-ud-train-doc3-1128
sv_lines-ud-train-doc6-2111
sv_lines-ud-train-doc6-2212
sv_lines-ud-train-doc6-2039
sv_lines-ud-train-doc7-2715
det tjänar ingenting till att springa till domaren om någon måste transporteras i ambulans till närmaste stad till exempel...
Joseph Asahe hade varit en av Edward Shinzas närmaste män i början av FFP:s tillvaro.
Käraste hälsningar. James.
Att borsta bort damm blev stans vanligaste gest.
Det fanns bara en mycket liten hög med silversiklar där inne och en endaste galleon av guld.


# Genetive Superlatives and Comparatives

In [8]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                                func1=lambda tok: (tok.form.lower().endswith('es') or tok.form.lower().endswith('as')),
                                                                                func2=lambda tok: tok.feats.get('Degree') in ['Sup', 'Cmp'])]
print(*list(set([tok.form for tok in matches])), sep='\n')
print(*list(set([tok.reconstruct_feats() for tok in matches])), sep='\n')
print(*list(set([tok.sent_id for tok in matches])), sep='\n')
print(*list(set([tok.sent.text for tok in matches])), sep='\n')


äldres
Case=Gen|Degree=Cmp
sv-ud-dev-485
sv-ud-test-1099
Skolbarnen bör från sina närbelägna fritidslokaler ha fritt tillträde till småsyskon och mindre kamrater, pensionärer må ha möjlighet att fraternisera nedom åldersstrecket, föräldrar må komma och gå - överhuvudtaget kan de äldres speciella kunskaper och intressen komma fler barn tillgodo.
(Jämför de äldres ställning i andra kulturer.)


# Form == Lemma

In [9]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                                func1=lambda tok: (not tok.form.endswith('åt') and
                                                                                                    not tok.form.endswith('rt') and
                                                                                                    not tok.form.endswith('kt') and
                                                                                                    not tok.form.endswith('lt') and
                                                                                                    not tok.form.endswith('pt') and
                                                                                                    not tok.form.endswith('nt') and
                                                                                                    not tok.form.endswith('ft') and
                                                                                                    not tok.form.endswith('igt') and
                                                                                                    not tok.form.endswith('st') and
                                                                                                    not tok.form.endswith('åst') and
                                                                                                    not tok.form.endswith('öst') and
                                                                                                    not tok.form.endswith('äst') and
                                                                                                    not tok.form.endswith('at') and
                                                                                                    not tok.form.endswith('ät') and
                                                                                                    not tok.form.endswith('öt') and
                                                                                                    not tok.form.endswith('et') and
                                                                                                    not tok.form.endswith('ut') and
                                                                                                    not tok.form.endswith('it') and
                                                                                                    not tok.form.endswith('tt') and
                                                                                                    not tok.form.endswith('a') and 
                                                                                                    not tok.form.endswith('ig') and 
                                                                                                    not tok.form.endswith('iv') and
                                                                                                    not tok.form.endswith('ös') and
                                                                                                    not tok.form.endswith('sam') and
                                                                                                    not tok.form.endswith('en') and
                                                                                                    not tok.form.endswith('nde') and
                                                                                                    not tok.form.endswith('sk') and
                                                                                                    not tok.form.endswith('är') and
                                                                                                    not tok.form.endswith('v') and
                                                                                                    not tok.form.endswith('l') and
                                                                                                    not tok.form.endswith('m') and
                                                                                                    not tok.form.endswith('ar') and
                                                                                                    not tok.form.endswith('er') and
                                                                                                    not tok.form.endswith('d') and
                                                                                                    not tok.form.endswith('n') and
                                                                                                    not tok.form.endswith('g') and
                                                                                                    not tok.form.endswith('i') and
                                                                                                    not tok.form.endswith('p') and
                                                                                                    not tok.form.endswith('k') and
                                                                                                    not tok.form.endswith('is') and
                                                                                                    not tok.form.endswith('rs') and
                                                                                                    not tok.form.endswith('es') and
                                                                                                    not tok.form.endswith('å') and
                                                                                                    not tok.form.endswith('r') and
                                                                                                    not tok.form.endswith('s') and
                                                                                                    not tok.form.endswith('ö') and
                                                                                                    not tok.form.endswith('e')),
                                                                                func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')


('30', 'Case=Nom|Number=Sing|NumType=Ord')
('1', 'Case=Nom|Number=Sing|NumType=Ord')
('12', 'Case=Nom|Number=Sing|NumType=Ord')
('22', 'Case=Nom|Number=Sing|NumType=Ord')
('3', 'Case=Nom|Number=Sing|NumType=Ord')
('4', 'Case=Nom|Number=Sing|NumType=Ord')
('4-5', 'Case=Nom|Number=Sing|NumType=Ord')
('29', 'Case=Nom|Number=Sing|NumType=Ord')
('III', 'Case=Nom|Number=Sing|NumType=Ord')
('snabb', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('Economic', 'Foreign=Yes')
('fräsch', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing')
('high', 'Foreign=Yes')
('Swedish', '_')
('North', '_')
('loj', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('redo', 'Definite=Ind|Degree=Pos')
('redo', 'Case=Nom|Degree=Pos')
('andino', 'Number=Sing')
('kortväxt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('New', 'Foreign=Yes')
('New', '_')
('New', 'Case=Nom')
('New', 'Degree=Pos')
('ny', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('splitterny', 'Case=Nom|

## -nt

In [15]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                                func1=lambda tok: (tok.form.endswith('nt') and
                                                                                                    not tok.form.endswith('ent') and
                                                                                                    not tok.form.endswith('ant')),
                                                                                func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('tungsint', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('styvsint', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('ont', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('runt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('gladlynt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('sällsynt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('närsynt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('kortsynt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('bevänt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing
Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing


In [16]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                                 lemma='rund')]
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')

('runda', 'Case=Nom|Definite=Def|Degree=Pos')
('runda', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Plur')
('rund', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('runt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
Case=Nom|Definite=Ind|Degree=Pos|Number=Plur
Case=Nom|Definite=Def|Degree=Pos
Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing
Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing


## -pt

In [18]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                                  func1=lambda tok: (tok.form.endswith('pt')),
                                                                                  func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('insvept', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing')
Case=Nom|Definite=Ind|Gender=Com|Number=Sing


## -ft

In [19]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                                    func1=lambda tok: (tok.form.endswith('ft')),
                                                                                    func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('gift', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('omgift', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('ogift', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('ogift', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('nygift', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing
Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing
Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part


## -lt

In [20]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: (tok.form.endswith('lt')),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('sacrococcygealt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('salt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('särskilt', 'Case=Nom|Definite=Ind|Gender=Neut|Number=Sing|Tense=Past|VerbForm=Part')
('stolt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('osmält', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part
Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing
Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing
Case=Nom|Definite=Ind|Gender=Neut|Number=Sing|Tense=Past|VerbForm=Part


## -!(nk|ck|sk|ak|ek|rk|sk)kt

In [21]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: (tok.form.endswith('kt') and
                                                                                      not tok.form.endswith('nkt') and
                                                                                      not tok.form.endswith('ckt') and
                                                                                      not tok.form.endswith('skt') and
                                                                                      not tok.form.endswith('akt') and
                                                                                      not tok.form.endswith('ekt') and
                                                                                      not tok.form.endswith('rkt') and
                                                                                      not tok.form.endswith('skt')),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('strikt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('ovikt', 'Case=Nom|Definite=Ind|Gender=Neut|Number=Sing|Tense=Past|VerbForm=Part')
('kokt', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('skalkokt', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('okokt', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('ihopläkt', 'Case=Nom|Definite=Ind|Gender=Neut|Number=Sing|Tense=Past|VerbForm=Part')
Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part
Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing
Case=Nom|Definite=Ind|Gender=Neut|Number=Sing|Tense=Past|VerbForm=Part


## -rt

In [22]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('rt') ,
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('godtagbart', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('smart', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('smart', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('svart', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('korpsvart', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('kort', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('kort', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('reaktionärt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
Case=Nom|Definite=Ind|Degree=Pos|Number=Sing
Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing
Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing


## -st

In [23]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('st') ,
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('fast', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('fast', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('handfast', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('högrest', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('trist', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('trist', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('robust', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('förtjust', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('förtjust', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('månbelyst', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('tyst', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('lättläst', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('uppblåst', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('svårlöst', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
Case=Nom|Definite=Ind|Degree=Pos|Number=S

## -ckt

In [24]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('ckt') ,
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('omtyckt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('dödsförskräckt', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('utsträckt', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
Case=Nom|Definite=Ind|Degree=Pos|Number=Sing
Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing


## -nkt

In [25]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('nkt') ,
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('sakrosankt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('oinskränkt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('genomtänkt', 'Case=Nom|Definite=Ind|Gender=Neut|Number=Sing|Tense=Past|VerbForm=Part')
('misstänkt', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('misstänkt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part
Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing
Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing
Case=Nom|Definite=Ind|Gender=Neut|Number=Sing|Tense=Past|VerbForm=Part


## -rkt

In [26]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: (tok.form.endswith('rkt')),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')


('dödsmärkt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('utmärkt', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('utmärkt', 'Case=Nom|Definite=Ind|Gender=Neut|Number=Sing|Tense=Past|VerbForm=Part')
('utmärkt', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('stärkt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('styvstärkt', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
Case=Nom|Definite=Ind|Degree=Pos|Number=Sing
Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing
Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part
Case=Nom|Definite=Ind|Gender=Neut|Number=Sing|Tense=Past|VerbForm=Part


## -(ol|yn|ys|ås|ös|äs)t

In [27]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: (tok.form.endswith('olt') or 
                                                                                      tok.form.endswith('ynt') or
                                                                                      tok.form.endswith('yst') or
                                                                                      tok.form.endswith('öst') or
                                                                                      tok.form.endswith('äst') or
                                                                                      tok.form.endswith('åst')),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')



('stolt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('gladlynt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('sällsynt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('närsynt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('kortsynt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('månbelyst', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('tyst', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('lättläst', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('uppblåst', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('svårlöst', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
Case=Nom|Definite=Ind|Degree=Pos|Number=Sing
Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing
Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing


## -skt

In [28]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('skt') ,
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('hemskt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('hemskt', '_')
Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing
_


## -igt

In [29]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: (tok.form.endswith('igt')),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('löjligt', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('husligt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('kvalmigt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('övrigt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
Case=Nom|Definite=Ind|Degree=Pos|Number=Sing
Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing


## -(an|en|ak|ek)t

In [30]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: (tok.form.endswith('ant') or
                                                                                      tok.form.endswith('ent') or
                                                                                      tok.form.endswith('akt') or
                                                                                      tok.form.endswith('akt')),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('abstrakt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('abstrakt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('exakt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('extravagant', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('elegant', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('arrogant', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('bekant', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('bekant', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('bekant', 'Case=Nom|Definite=Ind|Gender=Neut|Number=Sing')
('välbekant', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('obekant', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('brant', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('brant', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('intressant', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('intressant', 'Cas

## -it

In [31]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('it'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('vit', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('snövit', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing


## -ut

In [32]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('ut'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('akut', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('absolut', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('absolut', 'Case=Nom')
('slut', 'Case=Nom|Degree=Pos')
Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing
Case=Nom|Degree=Pos
Case=Nom


## -tt

In [33]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('tt'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('glatt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('platt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('fullsatt', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('fullsatt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('bosatt', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('typsatt', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('motsatt', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('motsatt', 'Case=Nom|Definite=Ind|Gender=Neut|Number=Sing|Tense=Past|VerbForm=Part')
('fortsatt', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('fortsatt', 'Case=Nom|Definite=Ind|Gender=Neut|Number=Sing|Tense=Past|VerbForm=Part')
('förutsatt', 'Case=Nom|Definite=Ind|Gender=Neut|Number=Sing|Tense=Past|VerbForm=Part')
('komplett', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('bortsett', 'Case=Nom|Definite=Ind|Gender=Neut|Number

## -at

In [34]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('at'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('separat', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('separat', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('moderat', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('desperat', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('privat', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('privat', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing
Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing


## -ät

In [35]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('ät'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('rät', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('lodrät', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('vågrät', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('tät', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing


## -åt

In [37]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('åt'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

## -öt

In [38]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('öt'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('blöt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('söt', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing


## -et

In [39]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('et'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('fet', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('grisfet', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('het', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('mycket', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('mycket', 'Degree=Pos')
('mycket', 'Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('mycket', 'Degree=Pos|Number=Sing')
('konkret', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('diskret', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('indiskret', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('litet', 'Degree=Pos')
('givet', 'Case=Nom|Definite=Def|Gender=Neut|Number=Sing')
Case=Nom|Definite=Def|Gender=Neut|Number=Sing
Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing
Case=Nom|Definite=Ind|Degree=Pos|Number=Sing
Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing
Degree=Pos
Degree=Pos|Number=Sing
Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing


## -:e

In [40]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith(':e'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('700:e', 'Case=Nom|Number=Sing|NumType=Ord')
('25:e', 'Case=Nom|Number=Sing|NumType=Ord')
Case=Nom|Number=Sing|NumType=Ord


## -!(nd|:)e
Här är det problem

In [41]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: (tok.form.endswith('e') and
                                                                                      not tok.form.endswith('nde') and
                                                                                      not tok.form.endswith(':e')),
                                                                   func2=lambda tok: tok.form == tok.lemma,
                                                                   func3=lambda tok: tok.feats.get('NumType', None) != 'Ord')]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('förenade', 'Case=Nom|Definite=Def|Degree=Pos|Number=Sing')
('öde', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('öde', 'Case=Nom|Definite=Def|Degree=Pos|Number=Sing')
('öde', 'Case=Nom')
('Free', '_')
('free', 'Degree=Pos')
('orange', 'Case=Nom|Degree=Pos|Gender=Neut|Number=Sing')
('gyllene', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('gyllene', 'Case=Nom|Definite=Def|Degree=Pos|Number=Sing')
('gyllene', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Plur')
('bakre', 'Case=Nom|Degree=Cmp')
('inre', 'Case=Nom|Degree=Cmp')
('yttre', 'Case=Nom|Degree=Pos|Number=Plur')
('yttre', 'Case=Nom|Degree=Cmp')
('övre', 'Case=Nom|Definite=Def|Degree=Pos')
('gängse', 'Case=Nom|Degree=Pos')
('ense', 'Case=Nom|Definite=Def|Degree=Pos|Number=Plur')
('varse', 'Case=Nom|Degree=Pos')
('diverse', 'Case=Nom|Definite=Ind|Degree=Pos')
('diverse', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Plur')
('lite', 'Degree=Pos')
('lite', 'Case=Nom')
('restante', 'Foreign=Yes')
('endaste', 'Case=Nom|Definite=Ind|Degre

## -ö

In [42]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('ö'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('slö', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing


## -å

In [43]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('å'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('få', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Plur')
('blå', 'Case=Nom|Degree=Pos|Number=Plur')
('blå', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Plur')
('blå', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('blå', 'Case=Nom|Definite=Def|Degree=Pos|Number=Sing')
('mörkblå', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('klarblå', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Plur')
('ljusblå', 'Case=Nom|Definite=Def|Degree=Pos|Number=Sing')
('ljusblå', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('rå', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('grå', 'Case=Nom|Definite=Def|Degree=Pos|Number=Sing')
('grå', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('rökgrå', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
Case=Nom|Degree=Pos|Number=Plur
Case=Nom|Definite=Def|Degree=Pos|Number=Sing
Case=Nom|Definite=Ind|Degree=Pos|Number=Plur
Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing
Case=Nom|Definite=Ind|Degree=Pos|Number=Sing


## -es

In [44]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('es'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('inbördes', 'Case=Nom|Degree=Pos')
('hes', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('inrikes', 'Case=Nom|Degree=Pos')
('urminnes', 'Case=Nom|Degree=Pos')
Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing
Case=Nom|Degree=Pos


## -rs

In [45]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('rs'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('stackars', 'Case=Nom|Definite=Def|Degree=Pos|Number=Sing')
('medelålders', 'Case=Nom')
Case=Nom|Definite=Def|Degree=Pos|Number=Sing
Case=Nom


## -is

In [46]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: (tok.form.endswith('is')),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('precis', 'Case=Nom')
('gratis', 'Case=Nom|Degree=Pos')
('envis', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('rättvis', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing
Case=Nom|Degree=Pos
Case=Nom


## -!(e|r|ö|i)s

In [47]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: (tok.form.endswith('s') and 
                                                                                      not tok.form.endswith('es') and
                                                                                      not tok.form.endswith('ös') and
                                                                                      not tok.form.endswith('rs')),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('vitas', 'Case=Gen|Definite=Def|Degree=Pos')
('svartas', 'Case=Gen|Definite=Def|Degree=Pos')
('gammaldags', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('precis', 'Case=Nom')
('gratis', 'Case=Nom|Degree=Pos')
('envis', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('rättvis', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('allsköns', 'Case=Nom|Degree=Pos')
('vass', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('viss', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('oviss', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('ljus', 'Case=Nom')
('ljus', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
Case=Nom
Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing
Case=Nom|Definite=Ind|Degree=Pos|Number=Sing
Case=Nom|Degree=Pos
Case=Gen|Definite=Def|Degree=Pos


## -!(e|a|ä)r

In [48]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: (tok.form.endswith('r') and 
                                                                                      not tok.form.endswith('er') and
                                                                                      not tok.form.endswith('ar') and
                                                                                      not tok.form.endswith('är')),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('skir', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('stor', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('jättestor', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('torr', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('dyr', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('svår', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('arbetsför', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing


## -i

In [49]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('i'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('fri', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('syndfri', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('skattefri', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('tullfri', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('priori', 'Foreign=Yes')
Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing
Foreign=Yes


## -p

In [50]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('p'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('tiptop', 'Case=Nom|Definite=Def|Degree=Pos|Number=Sing')
('slapp', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('skarp', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('djup', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing
Case=Nom|Definite=Def|Degree=Pos|Number=Sing


## -g

In [51]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('g'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('svag', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('seg', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('pigg', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('snygg', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('trygg', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('knubbig', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('höggradig', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('stadig', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('ostadig', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('suddig', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('smidig', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('tidig', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('enfaldig', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('skyldig', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('oskyldig', 'Case=Nom|Definite=Ind|Degree=P

## -n

In [52]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('n'),
                                                                   func3=lambda tok: not tok.form.endswith('en'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('American', 'Degree=Pos')
('American', '_')
('likadan', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('hurdan', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing|PronType=Int')
('sådan', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('European', 'Foreign=Yes')
('European', '_')
('profan', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('annan', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('fjärran', 'Case=Nom|Definite=Def|Degree=Pos')
('fjärran', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('ljusan', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('spontan', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('van', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('ovan', 'Case=Nom')
('ovan', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('fin', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('fin', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('jämn', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Co

## -k

In [53]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('k'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('elak', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('slak', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('rak', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('flack', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('tjock', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('läck', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('blek', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('vek', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('specifik', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('lik', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('lik', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('jättelik', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('drottninglik', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('jämlik', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('olik', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('sannol

## -d
Många VerbForm = Part

In [54]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('d'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('orubbad', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('laddad', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('skarpladdad', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('skyddad', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('värdeskyddad', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('halvskyddad', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('halvskyddad', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('inbäddad', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('avskedad', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('koleldad', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('obildad', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('utbildad', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('

## -l

In [55]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('l'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('global', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('katastrofal', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('social', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('personlig-social', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('genial', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('kal', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('radikal', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('lokal', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('minimal', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('maximal', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('gammal', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('urgammal', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('normal', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('normal', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('onormal', 'Ca

## -m

In [56]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('m'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('monogam', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('rättfram', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('stram', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('våldsam', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('långsam', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('blygsam', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('blygsam', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('tacksam', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('tveksam', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('verksam', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('yrkesverksam', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('ouppmärksam', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('stillsam', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('ensam', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('gemensam', 'Case=No

## -v

In [57]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('v'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('intensiv', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('kursiv', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('kursiv', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('aggressiv', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('negativ', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('auktoritativ', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('representativ', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('konservativ', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('positiv', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('aktiv', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('radioaktiv', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('interaktiv', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('effektiv', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('objektiv', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Si

## -ar

In [58]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('ar'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('användbar', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('godtagbar', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('gångbar', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('tänkbar', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('märkbar', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('valbar', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('omedelbar', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('ohållbar', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('uppenbar', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('underbar', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('underbar', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('dyrbar', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('genomförbar', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('jämförbar', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('hörbar', 'Ca

## -er


In [59]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('er'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print()
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
print('\n')
print(*list(set([tok.sent.text for tok in matches if (tok.form=='fler' and tok.feats.get('Degree', None) != 'Cmp') ])), sep='\n')
print('\n')
print(*list(set([tok.sent.text for tok in matches if (tok.form=='vänster' and tok.feats.get('Definite', None) is None) ])), sep='\n')

('söder', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('mager', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('mager', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('höger', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('vacker', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('vacker', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('säker', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('säker', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('osäker', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('fler', 'Case=Nom|Degree=Cmp')
('fler', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Plur')
('mer', 'Case=Nom|Degree=Cmp')
('bister', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('vänster', 'Case=Nom')
('vänster', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('dyster', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('väster', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Co

## -är

In [60]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('är'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('kär', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('prekär', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('skär', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('skär', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('primär', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('ordinär', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('preliminär', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('revolutionär', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('revolutionär', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('litterär', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('temporär', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('ekonomisk-monetär', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('militär', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('humanitär', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('elementär',

## -sk

In [61]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('sk'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('västerländsk', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('utländsk', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('oxfordsk', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('prosaisk', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('arabisk', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('juridisk', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('chassidisk', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('periodisk', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('judisk', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('europeisk', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('europeisk', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('östeuropeisk', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('magisk', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('tragisk', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('psykologisk-pedagogisk', 'Case=Nom|Def

## -iv

In [62]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('iv'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('intensiv', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('kursiv', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('kursiv', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('aggressiv', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('negativ', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('auktoritativ', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('representativ', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('konservativ', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('positiv', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('aktiv', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('radioaktiv', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('interaktiv', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('effektiv', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('objektiv', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Si

## -ig

In [63]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('ig'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('knubbig', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('höggradig', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('stadig', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('ostadig', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('suddig', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('smidig', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('tidig', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('enfaldig', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('skyldig', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('oskyldig', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('skattskyldig', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('väldig', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('väldig', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('myndig', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('fullmyndig', 'Case=Nom|De

## -ös (iös, lös)

In [64]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('ös'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('graciös', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('religiös', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('seriös', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('ambitiös', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('opretentiös', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('lös', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('ändlös', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('händelselös', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('sorglös', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('hemlös', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('tonlös', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('hopplös', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('hopplös', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('meningslös', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('uttryckslös', 'Case=N

## -sam

In [65]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('sam'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('våldsam', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('långsam', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('blygsam', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('blygsam', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('tacksam', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('tveksam', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('verksam', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('yrkesverksam', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('ouppmärksam', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('stillsam', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('ensam', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('gemensam', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('gynnsam', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('lönsam', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('mödosam', 'Case=No

## -en
Borde inte de ha verbet som lemma?

In [69]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('en'),
                                                                   func2=lambda tok: tok.form == tok.lemma,
                                                                   VerbForm='Part')]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.lemma, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('bunden', 'bunden', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('årstidsbunden', 'årstidsbunden', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('förbjuden', 'förbjuden', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('dragen', 'dragen', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('heldragen', 'heldragen', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('intagen', 'intagen', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('upptagen', 'upptagen', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('tvungen', 'tvungen', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('bibehållen', 'bibehållen', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('nytillkommen', 'nytillkommen', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('avsk

In [70]:
matches = talbanken_data.get_token_matches(upos='ADJ',
                                            func1=lambda tok: tok.form.endswith('en'),
                                            func2=lambda tok: tok.form == tok.lemma)
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('bunden', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('förbjuden', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('dragen', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('heldragen', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('upptagen', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('egen', 'Case=Nom|Degree=Pos|Gender=Com|Number=Sing')
('gedigen', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('mogen', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('naturtrogen', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('belägen', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('välbelägen', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('angelägen', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('benägen', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('vaken', 'Ca

In [71]:
matches = pud_data.get_token_matches(upos='ADJ',
                                            func1=lambda tok: tok.form.endswith('en'),
                                            func2=lambda tok: tok.form == tok.lemma)
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('årstidsbunden', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('intagen', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('egen', 'Case=Nom|Degree=Pos|Gender=Com|Number=Sing')
('tvungen', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('naken', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('avskuren', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('medveten', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('liten', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('kristen', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('skjuten', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('knuten', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('självbelåten', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('utgiven', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|Ver

In [72]:
matches = [tok for tb in [lines_data_dev, lines_data_test, lines_data_train] for tok in tb.get_token_matches(upos='ADJ',
                                                                                                             func1=lambda tok: tok.form.endswith('en'),
                                                                                                             func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('beriden', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('förvriden', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('förbjuden', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing')
('skräckslagen', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('lamslagen', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('uppslagen', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('uppslagen', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('storslagen', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('blåslagen', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('tillbakadragen', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('utdragen', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('upptagen', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('upptagen', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('självupptagen', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('egen', 'Case=Nom|Deg

## -nde

In [73]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   func1=lambda tok: tok.form.endswith('nde'),
                                                                   func2=lambda tok: tok.form == tok.lemma)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('ledande', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('vägledande', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('förberedande', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('gränsöverskridande', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('stridande', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('vanebildande', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('samhällsbildande', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('landande', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('självblandande', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('bindande', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('grundande', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('pensionsgrundande', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('bländande', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('barnavårdande', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('socialvårdande', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('motbjudande', 'Case=Nom|Deg

In [74]:
matches = talbanken_data.get_token_matches(upos='ADJ',
                                            func1=lambda tok: tok.form.endswith('ende'),
                                            func2=lambda tok: tok.form == tok.lemma)
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('förutseende', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('sammanboende', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('beroende', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('oberoende', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('troende', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('föregående', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('södergående', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('övergående', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('norrgående', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('långtgående', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('fortgående', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('vittgående', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('utgående', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('pågående', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('nedslående', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('enastående', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part

# Fixing Lemmas

## particip

### missing part-tag

In [80]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                                  func0=lambda tok: (tok.feats.get('VerbForm', None) != 'Part'),
                                                                                  func1=lambda tok: (tok.form.endswith('nde') or
                                                                                                     tok.form.endswith('en') or
                                                                                                     tok.form.endswith('t') or
                                                                                                     tok.form.endswith('d') or
                                                                                                     tok.form.endswith('dd') or
                                                                                                     tok.form.endswith('dda') or
                                                                                                     tok.form.endswith('tta') or
                                                                                                     tok.form.endswith('tt') or
                                                                                                     tok.form.endswith('na') or
                                                                                                     tok.form.endswith('de')))]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.lemma, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('stadda', 'stadd', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Plur')
('stadda', 'städja', 'Case=Nom|Degree=Pos|Number=Plur')
('härledda', 'härledd', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Plur')
('beredda', 'beredd', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Plur')
('ansedda', 'anse', 'Case=Nom|Definite=Def|Degree=Pos')
('baldakinförsedda', 'baldakinförsedd', 'Case=Nom|Definite=Def|Degree=Pos')
('spridda', 'spridd', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Plur')
('kringspridda', 'kringspridd', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Plur')
('tättbebodda', 'tättbebodd', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Plur')
('udda', 'udda', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Plur')
('udda', 'udda', 'Case=Nom|Degree=Pos')
('högljudda', 'högljudd', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Plur')
('rödklädda', 'rödklädd', 'Case=Nom|Definite=Def|Degree=Pos')
('skogklädda', 'skogklädd', 'Case=Nom|Definite=Def|Degree=Pos')
('svartklädda', 'svartklädd', 'Case=Nom|Definite=Ind|Degree=Pos|Numbe

In [21]:
matches = [(tok, key) for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                                         func0=lambda tok: (tok.feats.get('VerbForm', None) != 'Part'),
                                                                                         func1=lambda tok: ((tok.form.endswith('nde') or
                                                                                                            tok.form.endswith('en') or
                                                                                                            tok.form.endswith('t') or
                                                                                                            tok.form.endswith('d') or
                                                                                                            tok.form.endswith('dd') or
                                                                                                            tok.form.endswith('dda') or
                                                                                                            tok.form.endswith('tta') or
                                                                                                            tok.form.endswith('tt') or
                                                                                                            tok.form.endswith('na') or
                                                                                                            tok.form.endswith('de'))),
                                                                                         func3=lambda tok: (not tok.form == tok.lemma+'t'),
                                                                                         func4=lambda tok: (not tok.lemma.endswith('ig') and
                                                                                                            not tok.lemma.endswith('iv')),
                                                                                         func5=lambda tok: (tok.lemma != 'udda')
                                                                                         )]

                                                                                                            

with open('missing_part_feat.tsv', 'w') as f:
    examples = sorted(list(set([(tok.form, tok.lemma, tok.reconstruct_feats(), key) for tok, key in matches])), key=lambda a: a[0][::-1]) 
    features = sorted(list(set([(tok.reconstruct_feats(), key) for tok, key in matches])), key=lambda a: a[1][::-1])
    for form, lemma, feats, treebank in examples:
        f.write(f"{form=}\t{lemma=}\t{feats=}\t{treebank=}\n")
    f.write('\n')
    for feats, treebank in features:
        f.write(f"{feats=}\t{treebank=}\n")

### Presens

In [46]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   Tense='Pres',
                                                                   VerbForm='Part',
                                                                   func1=lambda tok: (not tok.lemma.endswith('nde')))]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.lemma, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[1][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('skyddande', 'skydda', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('ledande', 'leda', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('utredande', 'utreda', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('fortskridande', 'fortskrida', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('utbildande', 'utbilda', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('Tändande', 'tända', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('vårdande', 'vårda', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('förebyggande', 'förebygga', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('klarläggande', 'klarlägga', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('stigande', 'stiga', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('överstigande', 'överstiga', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('springande', 'springa', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('flygande', 'flyga', 'Case=Nom|Degree=Pos|Tense=Pres|VerbForm=Part')
('frågande', 'fråga', 'Case=Nom|Degree=Pos|Ten

### Perfekt

In [7]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   Tense='Past',
                                                                   VerbForm='Part',
                                                                   func1=lambda tok: (not tok.lemma.endswith('d') and
                                                                                      not tok.lemma.endswith('en') and
                                                                                      tok.lemma == tok.form))]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.lemma, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[1][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('sistnämnda', 'sistnämnda', 'Case=Nom|Number=Plur|Tense=Past|VerbForm=Part')
('sistnämnda', 'sistnämnda', 'Case=Nom|Definite=Def|Number=Sing|Tense=Past|VerbForm=Part')
('omvända', 'omvända', 'Case=Nom|Definite=Def|Number=Sing|Tense=Past|VerbForm=Part')
('använda', 'använda', 'Case=Nom|Number=Plur|Tense=Past|VerbForm=Part')
('gifta', 'gifta', 'Case=Nom|Number=Plur|Tense=Past|VerbForm=Part')
('icke-vita', 'icke-vita', 'Case=Nom|Definite=Def|Number=Sing|Tense=Past|VerbForm=Part')
('stadfästa', 'stadfästa', 'Case=Nom|Number=Plur|Tense=Past|VerbForm=Part')
('gift', 'gift', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('nygift', 'nygift', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('ovikt', 'ovikt', 'Case=Nom|Definite=Ind|Gender=Neut|Number=Sing|Tense=Past|VerbForm=Part')
('genomtänkt', 'genomtänkt', 'Case=Nom|Definite=Ind|Gender=Neut|Number=Sing|Tense=Past|VerbForm=Part')
('misstänkt', 'misstänkt', 'Case=Nom|Definite=Ind|Gender=Com|

In [5]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   Tense='Past',
                                                                   VerbForm='Part',
                                                                   func1=lambda tok: (not tok.lemma.endswith('a')
                                                                                      and not tok.lemma.endswith('d')
                                                                                      and not tok.lemma.endswith('en')))]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.lemma, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[1][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('angiven', 'ange', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('angivna', 'ange', 'Case=Nom|Definite=Def|Number=Sing|Tense=Past|VerbForm=Part')
('angivet', 'ange', 'Case=Nom|Definite=Ind|Gender=Neut|Number=Sing|Tense=Past|VerbForm=Part')
('angivna', 'ange', 'Case=Nom|Number=Plur|Tense=Past|VerbForm=Part')
('utgivet', 'utge', 'Case=Nom|Definite=Ind|Gender=Neut|Number=Sing|Tense=Past|VerbForm=Part')
('avgiven', 'avge', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('Sett', 'se', 'Case=Nom|Definite=Ind|Gender=Neut|Number=Sing|Tense=Past|VerbForm=Part')
('sett', 'se', 'Case=Nom|Definite=Ind|Gender=Neut|Number=Sing|Tense=Past|VerbForm=Part')
('frånsett', 'frånse', 'Case=Nom|Definite=Ind|Gender=Neut|Number=Sing|Tense=Past|VerbForm=Part')
('försedd', 'förse', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('försedda', 'förse', 'Case=Nom|Number=Plur|Tense=Past|VerbForm=Part')
('bortsett', 'bortse', 'Case=Nom|De

In [25]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   Tense='Past',
                                                                   VerbForm='Part',
                                                                   func1=lambda tok: tok.lemma.endswith('t'),
                                                                   func2=lambda tok: tok.lemma[-2] not in ['k', 't', 's', 'f'])]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.lemma, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('uttagna', 'ta_ut', 'Case=Nom|Number=Plur|Tense=Past|VerbForm=Part')
('slutet', 'slut', 'Case=Nom|Definite=Ind|Gender=Neut|Number=Sing|Tense=Past|VerbForm=Part')
('särskilt', 'särskilt', 'Case=Nom|Definite=Ind|Gender=Neut|Number=Sing|Tense=Past|VerbForm=Part')
('osmält', 'osmält', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part
Case=Nom|Number=Plur|Tense=Past|VerbForm=Part
Case=Nom|Definite=Ind|Gender=Neut|Number=Sing|Tense=Past|VerbForm=Part


In [34]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   Tense='Past',
                                                                   VerbForm='Part',
                                                                   func1=lambda tok: tok.lemma.endswith('en'))]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.lemma, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('igenkända', 'känna_igen', 'Case=Nom|Number=Plur|Tense=Past|VerbForm=Part')
('avlidna', 'avliden', 'Case=Nom|Number=Plur|Tense=Past|VerbForm=Part')
('bundna', 'bunden', 'Case=Nom|Number=Plur|Tense=Past|VerbForm=Part')
('bundna', 'bunden', 'Case=Nom|Definite=Def|Number=Sing|Tense=Past|VerbForm=Part')
('regelbundna', 'regelbunden', 'Case=Nom|Number=Plur|Tense=Past|VerbForm=Part')
('rollbundna', 'rollbunden', 'Case=Nom|Definite=Def|Number=Sing|Tense=Past|VerbForm=Part')
('obundna', 'obunden', 'Case=Nom|Number=Plur|Tense=Past|VerbForm=Part')
('respiratorbundna', 'respiratorbunden', 'Case=Nom|Definite=Def|Number=Sing|Tense=Past|VerbForm=Part')
('förbundna', 'förbunden', 'Case=Nom|Number=Plur|Tense=Past|VerbForm=Part')
('auktoritetsbundna', 'auktoritetsbunden', 'Case=Nom|Number=Plur|Tense=Past|VerbForm=Part')
('inbjudna', 'inbjuden', 'Case=Nom|Number=Plur|Tense=Past|VerbForm=Part')
('förbjudna', 'förbjuden', 'Case=Nom|Number=Plur|Tense=Past|VerbForm=Part')
('föreslagna', 'föreslagen', 'Case

In [40]:
matches = [(tok, key) for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                   Tense='Past',
                                                                   VerbForm='Part',
                                                                   func1=lambda tok: (tok.lemma.endswith('a')))]

with open('wrong_lemmas_a.tsv', 'w') as f:
    examples = sorted(list(set([(tok.form, tok.lemma, tok.reconstruct_feats(), key) for tok, key in matches])), key=lambda a: a[1][::-1]) 
    features = sorted(list(set([(tok.reconstruct_feats(), key) for tok, key in matches])), key=lambda a: a[1][::-1])
    for form, lemma, feats, treebank in examples:
        f.write(f"{form=}\t{lemma=}\t{feats=}\t{treebank=}\n")
    f.write('\n')
    for feats, treebank in features:
        f.write(f"{feats=}\t{treebank=}\n")

In [42]:

matches = [(tok, key) for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                                         Tense='Past',
                                                                                         VerbForm='Part',
                                                                                         func1=lambda tok: (not tok.lemma.endswith('a')
                                                                                                                              and not tok.lemma.endswith('d')
                                                                                                                              and not tok.lemma.endswith('en')
                                                                                                                              and not tok.lemma.endswith('satt')
                                                                                                                              and not (tok.lemma[-1] == 't' and tok.lemma[-2] in ['k', 'f', 's'])))]

with open('wrong_lemmas_rest.tsv', 'w') as f:
    examples = sorted(list(set([(tok.form, tok.lemma, tok.reconstruct_feats(), key) for tok, key in matches])), key=lambda a: a[1][::-1]) 
    features = sorted(list(set([(tok.reconstruct_feats(), key) for tok, key in matches])), key=lambda a: a[1][::-1])
    for form, lemma, feats, treebank in examples:
        f.write(f"{form=}\t{lemma=}\t{feats=}\t{treebank=}\n")
    f.write('\n')
    for feats, treebank in features:
        f.write(f"{feats=}\t{treebank=}\n")

In [47]:
matches = [(tok, key) for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                                         Tense='Pres',
                                                                                         VerbForm='Part',
                                                                                         func1=lambda tok: (not tok.lemma.endswith('nde')))]

with open('presensparticip_wrong_lemmas.tsv', 'w') as f:
    examples = sorted(list(set([(tok.form, tok.lemma, tok.reconstruct_feats(), key) for tok, key in matches])), key=lambda a: a[1][::-1]) 
    features = sorted(list(set([(tok.reconstruct_feats(), key) for tok, key in matches])), key=lambda a: a[1][::-1])
    for form, lemma, feats, treebank in examples:
        f.write(f"{form=}\t{lemma=}\t{feats=}\t{treebank=}\n")
    f.write('\n')
    for feats, treebank in features:
        f.write(f"{feats=}\t{treebank=}\n")

In [49]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                                  func0=lambda tok: tok.feats.get('VerbForm', None) != 'Part',
                                                                                  func1=lambda tok: (tok.lemma.endswith('nde') or
                                                                                                     tok.lemma.endswith('ndes') or
                                                                                                     tok.lemma.endswith('ad') or
                                                                                                     tok.lemma.endswith('en')))]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.lemma, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[1][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('skrivskyddat', 'skrivskyddad', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('halvskyddad', 'halvskyddad', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('koleldad', 'koleldad', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('obildad', 'obildad', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('inblandat', 'inblandad', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('inblandade', 'inblandad', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Plur')
('oblandad', 'oblandad', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('hopblandad', 'hopblandad', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('sabeltandad', 'sabeltandad', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('fulländade', 'fulländad', 'Case=Nom|Definite=Def|Degree=Pos|Gender=Com|Number=Sing')
('vårdad', 'vårdad', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing')
('förlegad', 'förlegad', 'Case=Nom|Definite=Ind|Degree=Pos|

### VerbForm=Part|Tense=Past -> Lemma=Form?

In [5]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                                  VerbForm='Part',
                                                                                  Tense='Past',
                                                                                  func1=lambda tok: tok.lemma != tok.form)]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.lemma, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[1][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('dubbade', 'dubba', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Plur|Tense=Past|VerbForm=Part')
('avtrubbade', 'avtrubba', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Plur|Tense=Past|VerbForm=Part')
('skadad', 'skada', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('skadade', 'skada', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Plur|Tense=Past|VerbForm=Part')
('skadade', 'skada', 'Case=Nom|Number=Plur|Tense=Past|VerbForm=Part')
('Skyddad', 'skydda', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('skyddade', 'skydda', 'Case=Nom|Definite=Def|Degree=Pos|Tense=Past|VerbForm=Part')
('beskyddad', 'beskydda', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('överbeskyddade', 'överbeskydda', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Plur|Tense=Past|VerbForm=Part')
('ledd', 'leda', 'Case=Nom|Definite=Ind|Gender=Com|Number=Sing|Tense=Past|VerbForm=Part')
('ledda', 'leda', 'Case=Nom|Number=Plur|

## Checking if there are cases where Def should have number

In [44]:
matches = [(tok, key) for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                                  Definite='Def',
                                                                                  func2=lambda tok: tok.form.endswith('a')
                                                                                  )]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.lemma, tok.reconstruct_feats(), key) for tok, key in matches])), key=lambda a: a[1][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok, key in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('snedvridna', 'snedvrida', 'Case=Nom|Definite=Def|Tense=Past|VerbForm=Part', 'talbanken')
('sistnämnda', 'sistnämnda', 'Case=Nom|Definite=Def|Tense=Past|VerbForm=Part', 'talbanken')
('omvända', 'omvända', 'Case=Nom|Definite=Def|Tense=Past|VerbForm=Part', 'talbanken')
('planlagda', 'planlägga', 'Case=Nom|Definite=Def|Degree=Pos', 'lines_test')
('sammanlagda', 'sammanlägga', 'Case=Nom|Definite=Def|Tense=Past|VerbForm=Part', 'talbanken')
('tvåpoliga', 'tvåpoliga', 'Case=Nom|Definite=Def|Degree=Pos', 'pud')
('nyliga', 'nyliga', 'Case=Nom|Definite=Def|Degree=Pos', 'pud')
('förlängda', 'förlänga', 'Case=Nom|Definite=Def|Tense=Past|VerbForm=Part', 'talbanken')
('dolda', 'dölja', 'Case=Nom|Definite=Def|Tense=Past|VerbForm=Part', 'lines_test')
('dolda', 'dölja', 'Case=Nom|Definite=Def|Tense=Past|VerbForm=Part', 'talbanken')
('avröjda', 'avröja', 'Case=Nom|Definite=Def|Degree=Pos|Gender=Com', 'lines_test')
('undertryckta', 'undertrycka', 'Case=Nom|Definite=Def|Degree=Pos|Gender=Com', 'lines_tes

In [48]:
matches = [(tok, key) for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                                  Definite='Def',
                                                                                  func1=lambda tok: tok.feats.get('Number', None) is not None,
                                                                                  func2=lambda tok: not tok.form.endswith('a') and not tok.form.lower().endswith('e')
                                                                                  )]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.lemma, tok.reconstruct_feats(), tok.sent.text, key) for tok, key in matches])), key=lambda a: a[1][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok, key in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('enskildes', 'enskild', 'Case=Gen|Definite=Def|Degree=Pos|Gender=Masc|Number=Sing', 'Arbetstakten och arbetsförhållandena anpassas efter den enskildes handikapp.', 'talbanken')
('enskildes', 'enskild', 'Case=Gen|Definite=Def|Degree=Pos|Gender=Masc|Number=Sing', 'Där har också den enskildes läggning och intressen kommit att värderas mera, vilket ofta medför att familjens medlemmar splittras också i fråga om nöjen och i fråga om religiöst engagemang.', 'talbanken')
('sjukes', 'sjuk', 'Case=Gen|Definite=Def|Degree=Pos|Gender=Masc|Number=Sing', 'Eftersom den har en mera förödande effekt på den sjukes själsliv, är den ännu allvarligare.', 'talbanken')
('andres', 'annan', 'Case=Gen|Definite=Def|Degree=Pos|Gender=Masc|Number=Sing', "Det är frestande nog att gömma undan det som riktigt är jag själv under den andres dominans.' 1", 'talbanken')
('avlidnes', 'avliden', 'Case=Gen|Definite=Def|Gender=Masc|Number=Sing|Tense=Past|VerbForm=Part', 'Kanske har hus- och bilköp skett i den avlidnes namn,

## Checking if Ind should have Plur

In [22]:
matches = [(tok, key) for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                                  Definite='Ind',
                                                                                  func1=lambda tok: tok.feats.get('Number', None) != 'Plur',
                                                                                  func2=lambda tok: tok.form.endswith('a')
                                                                                  )]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.lemma, tok.reconstruct_feats(), key) for tok, key in matches])), key=lambda a: a[1][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok, key in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('enahanda', 'enahanda', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing', 'lines_dev')
('enahanda', 'enahanda', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing', 'lines_test')
('jäkla', 'jäkla', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing', 'lines_test')
('stilla', 'stilla', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing', 'lines_test')
('stilla', 'stilla', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing', 'lines_dev')
('jävla', 'jävla', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing', 'lines_test')
('samma', 'samma', 'Definite=Ind|PronType=Ind', 'lines_dev')
('samma', 'samma', 'Definite=Ind|PronType=Ind', 'lines_test')
('bästa', 'bra', 'Case=Nom|Definite=Ind|Degree=Sup', 'lines_test')
('rosa', 'rosa', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing', 'lines_dev')
('laxrosa', 'laxrosa', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Com|Number=Sing', 'lines_test')
('sista', 'sista', 'Case=Nom|Definite=Ind|Degree=Sup', 'lines_test')
('sista', 'sista', 'Case=Nom|Definite=Ind|De

We cannot assume that Nom, Pos, Plur is Ind, because there are cases in the treebank were Def have been classified with these features.

In [18]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                                  Case='Nom',
                                                                                  Degree='Pos',
                                                                                  Number='Plur',
                                                                                  func1=lambda tok: tok.feats.get('Definite', None) != 'Ind',
                                                                                  #func2=lambda tok: tok.form.endswith('a')
                                                                                  )]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.lemma, tok.reconstruct_feats(), tok.sent.text) for tok in matches])), key=lambda a: a[1][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')
print(len(matches))

('rädda', 'rädda', 'Case=Nom|Degree=Pos|Number=Plur', 'Motståndarna är ofta av någon anledning rädda för hundar.')
('båda', 'båda', 'Case=Nom|Degree=Pos|Number=Plur', 'I denna, säger han, fördelas rollerna ganska fritt och efter vad som passar båda parterna.')
('båda', 'båda', 'Case=Nom|Degree=Pos|Number=Plur', 'Familjer med två barn eller flera kan få båda beloppen.')
('båda', 'båda', 'Case=Nom|Degree=Pos|Number=Plur', 'Det gamla knepet att köra upp bilen mot garageväggen och kolla ljuset är värdelöst; då får du möjligen veta om båda strålkastarna lyser, men inte mycket mera.')
('båda', 'båda', 'Case=Nom|Degree=Pos|Number=Plur', 'Ska man svänga till vänster på en väg eller gata med fordonstrafik i båda riktningarna, ska man ligga strax till höger om vägens mitt oavsett om det finns ett eller flera körfält i färdriktningen.')
('båda', 'båda', 'Case=Nom|Degree=Pos|Number=Plur', 'Den kan sökas hos domstolen av båda makarna tillsammans.')
('båda', 'båda', 'Case=Nom|Degree=Pos|Number=Plur'

## normal adj

In [6]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                                  func0=lambda tok: tok.feats.get('VerbForm', None) != 'Part')]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.lemma, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[1][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('graecas', 'graeca*', '_')
('s.k.', 's.k.', 'Abbr=Yes')
('Kungl.', 'Kungl.', 'Abbr=Yes')
('St.', 'St.', 'Abbr=Yes')
('110:e', '110', 'Case=Nom')
('tjugonde', '20', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('tjugonde', '20', 'Case=Nom')
('30', '30', 'Case=Nom')
('70:e', '70', 'Case=Nom')
('1', '1', 'Case=Nom')
('12', '12', 'Case=Nom')
('112:e', '112', 'Case=Nom')
('22', '22', 'Case=Nom')
('72:a', '72', 'Case=Nom')
('3', '3', 'Case=Nom')
('4', '4', 'Case=Nom')
('4-5', '4-5', 'Case=Nom')
('45:e', '45', 'Case=Nom')
('116:e', '116', 'Case=Nom')
('107:e', '107', 'Case=Nom')
('97:e', '97', 'Case=Nom')
('119:e', '119', 'Case=Nom')
('29', '29', 'Case=Nom')
('99:e', '99', 'Case=Nom|Definite=Def|Degree=Pos|Number=Sing')
('III', 'III', 'Case=Nom|Number=Sing|NumType=Ord')
('udda', 'udda', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Plur')
('udda', 'udda', 'Case=Nom|Degree=Pos')
('skyddade', 'skydda', 'Case=Nom|Definite=Def|Degree=Pos|Number=Sing')
('skyddade', 'skydda', 'Case=Nom|D

In [10]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                                  func0=lambda tok: tok.feats.get('VerbForm', None) != 'Part',
                                                                                  func1=lambda tok: (tok.lemma[-1] == 't' and
                                                                                                    tok.lemma[-2] not in ['s', 't', 'f', 'k', 'p']) or
                                                                                                    tok.lemma.endswith('a'))]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.lemma, tok.reconstruct_feats()) for tok in matches])), key=lambda a: a[1][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('udda', 'udda', 'Case=Nom|Degree=Pos')
('udda', 'udda', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Plur')
('skyddade', 'skydda', 'Case=Nom|Definite=Def|Degree=Pos|Number=Sing')
('skyddade', 'skydda', 'Case=Nom|Definite=Def|Degree=Pos|Gender=Com|Number=Sing')
('rädda', 'rädda', 'Case=Nom|Degree=Pos|Number=Plur')
('berett', 'bereda', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('medeltida', 'medeltida', 'Case=Nom|Degree=Pos')
('framtida', 'framtida', 'Case=Nom|Degree=Pos')
('samtida', 'samtida', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Plur')
('samtida', 'samtida', 'Case=Nom|Degree=Pos')
('förtida', 'förtida', 'Case=Nom|Degree=Pos')
('Förtida', 'förtida', 'Case=Nom|Degree=Pos')
('nutida', 'nutida', 'Case=Nom|Degree=Pos')
('enahanda', 'enahanda', 'Case=Nom|Definite=Ind|Degree=Pos|Number=Sing')
('enahanda', 'enahanda', 'Case=Nom|Definite=Ind|Degree=Pos|Gender=Neut|Number=Sing')
('blandade', 'blanda', 'Case=Nom|Definite=Def|Degree=Pos|Number=Sing')
('blandat', 'blanda', 'Case

In [ ]:
matches = [(tok, key) for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                                         func0=lambda tok: tok.feats.get('VerbForm', None) != 'Part')]

with open('normal_adj_wrong_lemmas.tsv', 'w') as f:
    examples = sorted(list(set([(tok.form, tok.lemma, tok.reconstruct_feats(), key) for tok, key in matches])), key=lambda a: a[1][::-1]) 
    features = sorted(list(set([(tok.reconstruct_feats(), key) for tok, key in matches])), key=lambda a: a[1][::-1])
    for form, lemma, feats, treebank in examples:
        f.write(f"{form=}\t{lemma=}\t{feats=}\t{treebank=}\n")
    f.write('\n')
    for feats, treebank in features:
        f.write(f"{feats=}\t{treebank=}\n")

In [81]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                                  Abbr='Yes')]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.lemma, tok.reconstruct_feats(), tok.sent.text) for tok in matches])), key=lambda a: a[1][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('s k', 'så_kallad', 'Abbr=Yes|Case=Nom|Degree=Pos', 'Det är för övrigt samma operationsmetod som användes hos barn som föds med s k vattenskalle.')
('s k', 'så_kallad', 'Abbr=Yes|Case=Nom|Degree=Pos', '18 afrikanska stater och Madagaskar plus Grekland och Turkiet är nu knutna till EEC genom s k associationsavtal.')
('s.k.', 'så_kallad', 'Abbr=Yes|Case=Nom|Degree=Pos', 'För dem tycks inte heller de s.k. typiska kvinnoyrkena vara ett alltför svårt hinder.')
('s k', 'så_kallad', 'Abbr=Yes|Case=Nom|Degree=Pos', 'Det stadgas i den s k stenkolslagen från 1886 som dammats av för den här situationen.')
('s k', 'så_kallad', 'Abbr=Yes|Case=Nom|Degree=Pos', 'Inte heller ville han att EEC skulle bli ett s k överstatligt organ genom vilket medlemsländerna helt kunde styras.')
('s k', 'så_kallad', 'Abbr=Yes|Case=Nom|Degree=Pos', 'Ändrade förmåner vid s k halvöppen vård.')
('s k', 'så_kallad', 'Abbr=Yes|Case=Nom|Degree=Pos', 'Om en sådan påfrestning blir långvarig, kan resultatet bli ett tillstånd a

In [82]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                                  func1=lambda tok: (tok.form.endswith('as')),
                                                                                  func2=lambda tok: tok.feats.get('VerbForm', None) is None
                                                                                  )]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.lemma, tok.reconstruct_feats(), tok.sent.text) for tok in matches])), key=lambda a: a[1][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('graecas', 'graeca*', 'Case=MANUAL_FIX|Degree=Pos', 'För det andra: Vi bör inte tillåta någon förskjutning ad calendas graecas och inte införa någon inskränkning till fysiska personer.')
('frånskildas', 'frånskild', 'Case=Gen|Degree=Pos|Number=Plur', 'Till realismen hör att i själavården ta hänsyn till t.ex. frånskildas och änkors / änklingars rent sexuella nöd.')
('färgblindas', 'färgblind', 'Case=Gen|Degree=Pos|Number=Plur', 'Till färgblindas hjälp får de också olika streckmarkeringar.')
('dödas', 'död', 'Case=Gen|Definite=Def|Degree=Pos', 'De dödas härskaror i alla riktningar, ändlösa.')
('dödas', 'död', 'Case=Gen|Definite=Def|Degree=Pos', 'Varpå en ung man med hatten ner mot ena ögonbrynet, kontorist skulle jag tro – det måste ju ha funnits en mängd personal i företaget fast hela huset var tyst som en de dödas boning – kom ner från de övre regionerna och tog mig med sig.')
('omyndigas', 'omyndig', 'Case=Gen|Definite=Ind|Degree=Pos|Number=Plur', 'Som ni vet har omyndiga trollkarlar

### -e form gender=Masc

In [86]:
matches = [tok for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADJ',
                                                                                  Gender='Masc')]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.lemma, tok.reconstruct_feats(), tok.sent.text) for tok in matches])), key=lambda a: a[1][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('ene', 'ena', 'Case=Nom|Definite=Def|Degree=Pos|Gender=Masc|Number=Sing', 'Varför skulle den ene föredra barn och den andre yrkesarbete?')
('ene', 'ena', 'Case=Nom|Definite=Def|Degree=Pos|Gender=Masc|Number=Sing', "Den ene får kanske ett 'bättre' jobb än den andre.")
('enskilde', 'enskild', 'Case=Nom|Definite=Def|Degree=Pos|Gender=Masc|Number=Sing', 'I morgondagens socialpolitik blir det inte nog bara med att trygga rätten till ett existensminimum för den enskilde.')
('enskilde', 'enskild', 'Case=Nom|Definite=Def|Degree=Pos|Gender=Masc|Number=Sing', 'Det kan formuleras så olika - men i varje fall vet vi att det inte räcker med att barn är mätta och rena och sysselsatta - saker som i och för sej kan hålla den enskilde vårdaren fullt sysselsatt.')
('enskilde', 'enskild', 'Case=Nom|Definite=Def|Degree=Pos|Gender=Masc|Number=Sing', 'Brottspåföljder bör avpassas så att den enskilde lagöverträdaren avhåller sig från att begå nya brott och återinpassas i ett normalt samhällsliv.')
('enskilde

In [21]:
matches = [(tok, key) for (key, tb) in treebanks.items() for tok in tb.get_token_matches(upos='ADV',
                                                                                         func1=lambda tok: (tok.feats.get('Degree', None) == 'Cmp' or tok.feats.get('Degree', None) == 'Sup'))]
# print(*list(set([tok.form for tok in matches])), sep='\n')
print(*sorted(list(set([(tok.form, tok.lemma, tok.reconstruct_feats(), key) for tok, key in matches])), key=lambda a: a[1][::-1]), sep='\n')
print(*sorted(list(set([tok.reconstruct_feats() for tok, key in matches])), key=lambda a: a[0][::-1]), sep='\n')
# print(*list(set([tok.sent_id for tok in matches])), sep='\n')
# print(*list(set([tok.sent.text for tok in matches])), sep='\n')

('värst', 'illa', 'Degree=Sup', 'pud')
('sämre', 'illa', 'Degree=Cmp', 'talbanken')
('Hellre', 'gärna', 'Degree=Cmp', 'talbanken')
('hellre', 'gärna', 'Degree=Cmp', 'pud')
('helst', 'gärna', 'Degree=Sup', 'talbanken')
('helst', 'gärna', 'Degree=Sup', 'pud')
('hellre', 'gärna', 'Degree=Cmp', 'talbanken')
('Bäst', 'bra', 'Degree=Sup', 'talbanken')
('bättre', 'bra', 'Degree=Cmp', 'talbanken')
('bättre', 'bra', 'Degree=Cmp', 'lines_test')
('bättre', 'bra', 'Degree=Cmp', 'pud')
('BÄTTRE', 'bra', 'Degree=Cmp', 'lines_test')
('bäst', 'bra', 'Degree=Sup', 'talbanken')
('bäst', 'bra', 'Degree=Sup', 'lines_test')
('bättre', 'bra', 'Degree=Cmp', 'lines_dev')
('närmast', 'nära', 'Degree=Sup', 'lines_test')
('närmare', 'nära', 'Degree=Cmp', 'lines_test')
('närmare', 'nära', 'Degree=Cmp', 'pud')
('Närmast', 'nära', 'Degree=Sup', 'talbanken')
('närmare', 'nära', 'Degree=Cmp', 'lines_dev')
('närmast', 'nära', 'Degree=Sup', 'lines_dev')
('närmast', 'nära', 'Degree=Sup', 'talbanken')
('närmare', 'nära',